# 論文修正用、D2NNの学習

In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
!git clone https://github.com/konnitiha3/D2NN.git

import sys
sys.path.append('/content/D2NN')

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import time
import os
import json
import sys

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from module.two_dim.layers import *

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [3]:
#@title デフォルトのタイトル テキスト
wavelength = 633.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}
n = 1.5 #@param {type:"number"}

In [4]:
def create_model(**kwargs):
    shape = (100, 100)
    inputs = tf.keras.Input((28, 28))
    phi_max = 2.0 * np.pi
    x = ImageResizing(shape)(inputs)
    x = ImageBinarization(0.5, 0.0, 1.0)(x)
    x = IntensityToElectricField()(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = Modulation(limitation='sin', phi_max=phi_max)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.7e-3, d=d, n=n, method="expand")(x)
    x = ElectricFieldToIntensity(normalization="max")(x)
    x = MNISTDetector(10)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [ ]:
model = create_model()
model.summary()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
    )

epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    min_delta=0.05,
    patience=2,
)

model_name = "20220608_1"
cholab_path = "/content/drive/MyDrive/D2NN/"
checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

result = model.fit(x_train,
                   y_train,
                   batch_size=64,
                   epochs=epochs,
                   validation_data=(x_test, y_test),
                   callbacks=[cp_callback]
                   )

#path = cholab_path + "trained_model/"+ model_name
path = 'trained_model/test'
model.save(path)

df = pd.DataFrame(result.history)
df.to_csv(path + "/history.csv")

df = pd.DataFrame(result.history)
df.to_csv(path + "/history.csv")

with open(path + "/config.json", 'w') as f:
    json.dump(model.get_config(), f, indent=4)